In [16]:
import inseq
import pandas as pd
import warnings
import byt5_model
import torch

warnings.filterwarnings("ignore")

In [12]:
!pwd

/Users/auke/nlp-project-group-4


In [34]:
class InseqAttributer:
    def __init__(self, model="./models/test/", attribution_method="input_x_gradient") -> None:
        """Create an object that loads a given model through Inseq with a given attribution method"""
        self.model = inseq.load_model(model, attribution_method)
    
    def attribute(self, inp:str, out:str=None):
        """Use the Inseq model to generate feature attributions using a given input or input & output"""
        if inp and out:
            inp_out = (inp, out)
        else:
            inp_out = (inp,)

        inseq_out = self.model.attribute(*inp_out,
                                         attribute_target=True,
                                         step_scores=["probability"],
                                         n_steps=300
                                        )
        inseq_out.show()


def predict_on_data(data, model_config_path, spaces=True, device='auto'):
    if device == 'auto':
        device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    elif device == 'gpu' or device == 'cuda':
        device = torch.device("cuda")
    elif device == 'cpu':
        device = torch.device("cpu")
    else:
        print('Set device to auto, cpu, or gpu/cuda.')
        return


    test = data
    if spaces:
        test['inputs'] = test['lemma'] + ' ' + test['features']
    else:
        test['inputs'] = test['lemma'] + test['features']


    test_gen_comparison = byt5_model.comparer(test, byt5_model.generate(model_config_path, test, device))
    test = pd.concat([test, test_gen_comparison], axis=1).rename(columns={'labels': 'plural'})

    correct = []
    for idx, row in test.iterrows():
        if row['Expected'] == row['Predicted']:
            correct.append('correct')
        else:
            correct.append('incorrect')

    test = test.assign(correct=correct)

    return pd.concat([test[test.correct == 'incorrect'],
                      test[test.correct == 'correct']])


def inseq_on_predictions(samples, inseq_model, expected=False):
    count = 0
    for idx, sample in samples.iterrows():
        count += 1
        if count > 1:
            print('\n')
        print('='*80)

        inp = sample['lemma'] + ' ' + sample['features']
        if expected:
            print('Testing Expected (gold) outcome (what would the model look at to get to the expected outcome)')
            output_type = 'gold'
            out = sample['Expected']
        else:
            print('Testing Predicted (pred) outcome (what did the model take from the input during generation)')
            output_type = 'pred'
            out = sample['Predicted']

        print('Sample: {}\nIndex: {}\nThis prediction is {}\nInput -> output ({}): \'{}\' -> \'{}\''.format(
             count, idx, sample['correct'], output_type, inp, out
        ))
        if output_type == 'pred':
            print(f"The expected or gold output would be '{sample['Expected']}'")

        print('='*80)

        inseq_model.attribute(inp,
                              out)

## Turkish

Focus on relation between last vowel of the stem and the first vowel of the suffix. We expect the last vowel to be salient in predicting the first vowel of the suffix. We focus on plural nouns.

In [7]:
model_tur_finetuned = "./drive-symlink/NLP_project_morphological_inflection/finetuned_tur_1"
inseq_tur_finetuned = InseqAttributer(model_tur_finetuned, "input_x_gradient")

In [8]:
sample_runner(data_sampler('./data/tur.gold', model_tur_finetuned, device='cpu'), inseq_tur_finetuned, expected=True)

                      Expected                       Predicted
0        satıyor olmalı mıydık           satıyor olmalı mıydın
1                hamızlarından                  hamızlarınızda
2                  ovmaz mıydı          ovuyor olmamalı mıydın
3     yıpranmamalı mıymışsınız         yıpranmış olacak mıydık
4                   hüzünlerde                    hüzünlerinde
5                manivelanızda              manivelalarınızdan
6            dokuyor olmalısın            dokumuş olacak mıydı
7         barışmamalılar mıydı       barışıyor olmamalı mıydık
8      sahipli yazılımlarımızı             sahipli yazılımızda
9            yanıt vermedim mi           yanıt vermemeli miydi
10       onayacak olacak mısın                 onamamış mıydık
11          tıklamış olacaktın               tıklamamış mıydın
12              prostatınızdan                prostatlarınızda
13                damlarlar mı               damlamamış mıydık
14  hareket ediyor olacaktınız         hareket etmemeli

Attributing with input_x_gradient...: 100%|█████████████████████████████| 26/26 [00:21<00:00,  1.14it/s]


,a,t,Ä,±,l,m,Ä,±,Å,,,o,l,m,a,y,a,c,a,k,s,Ä,±,n,</s>
a,0.04,0.035,0.039,0.022,0.029,0.019,0.046,0.032,0.033,0.025,0.023,0.034,0.009,0.013,0.026,0.021,0.015,0.01,0.039,0.029,0.022,0.036,0.023,0.014,0.021
t,0.082,0.054,0.038,0.034,0.055,0.028,0.029,0.025,0.03,0.024,0.038,0.042,0.026,0.03,0.029,0.023,0.024,0.011,0.03,0.012,0.01,0.01,0.021,0.013,0.017
Ä,0.038,0.048,0.076,0.04,0.04,0.048,0.026,0.021,0.029,0.033,0.02,0.019,0.017,0.016,0.028,0.032,0.015,0.014,0.009,0.016,0.028,0.02,0.013,0.029,0.019
±,0.052,0.042,0.071,0.048,0.044,0.034,0.075,0.035,0.043,0.018,0.016,0.021,0.014,0.024,0.02,0.018,0.02,0.022,0.008,0.027,0.014,0.021,0.025,0.013,0.018
l,0.061,0.066,0.063,0.07,0.055,0.029,0.022,0.02,0.033,0.025,0.032,0.015,0.03,0.029,0.03,0.026,0.012,0.015,0.012,0.025,0.011,0.018,0.018,0.018,0.022
m,0.015,0.017,0.017,0.02,0.024,0.039,0.036,0.014,0.019,0.014,0.016,0.013,0.014,0.019,0.034,0.026,0.025,0.012,0.008,0.016,0.011,0.011,0.016,0.014,0.01
a,0.015,0.029,0.027,0.02,0.025,0.017,0.028,0.017,0.019,0.017,0.015,0.022,0.017,0.01,0.037,0.02,0.024,0.014,0.014,0.016,0.008,0.011,0.012,0.007,0.012
k,0.035,0.038,0.025,0.022,0.03,0.057,0.037,0.03,0.021,0.028,0.032,0.023,0.013,0.018,0.029,0.026,0.025,0.017,0.019,0.023,0.021,0.02,0.017,0.02,0.018
,0.057,0.043,0.031,0.024,0.036,0.035,0.017,0.031,0.017,0.016,0.033,0.01,0.014,0.01,0.025,0.045,0.014,0.027,0.011,0.02,0.017,0.017,0.017,0.011,0.01
V,0.013,0.012,0.013,0.009,0.015,0.012,0.01,0.008,0.008,0.007,0.011,0.007,0.007,0.009,0.012,0.011,0.006,0.006,0.008,0.012,0.009,0.008,0.01,0.007,0.01




Testing Expected (gold) outcome (what would the model look at to get to the expected outcome)
Sample: 2
Index: 750
This prediction is incorrect
Input -> output (gold): 'katılmak V;OBLIG;SG;3;NEG;PRS;INTR' -> 'katılmameli mi'


Attributing with input_x_gradient...: 100%|█████████████████████████████| 17/17 [00:10<00:00,  1.58it/s]


...

Attributing with input_x_gradient...: 100%|█████████████████████████████| 14/14 [00:06<00:00,  2.11it/s]


,i,l,k,e,l,e,r,i,n,d,e,n,</s>
i,0.073,0.1,0.145,0.167,0.069,0.035,0.079,0.081,0.07,0.141,0.032,0.033,0.145
l,0.197,0.082,0.092,0.059,0.104,0.077,0.071,0.065,0.037,0.066,0.028,0.089,0.038
k,0.051,0.077,0.063,0.044,0.056,0.06,0.074,0.089,0.021,0.045,0.076,0.029,0.088
e,0.121,0.058,0.132,0.148,0.072,0.024,0.03,0.037,0.069,0.026,0.035,0.075,0.058
,0.045,0.043,0.018,0.029,0.143,0.062,0.04,0.032,0.031,0.018,0.022,0.019,0.019
N,0.029,0.047,0.073,0.047,0.031,0.052,0.031,0.048,0.018,0.019,0.047,0.017,0.02
;,0.046,0.077,0.029,0.034,0.044,0.027,0.034,0.024,0.046,0.027,0.025,0.02,0.028
A,0.016,0.032,0.024,0.015,0.024,0.016,0.016,0.018,0.01,0.012,0.014,0.007,0.014
B,0.032,0.072,0.057,0.059,0.043,0.03,0.014,0.013,0.03,0.017,0.01,0.015,0.006
L,0.013,0.028,0.066,0.051,0.011,0.013,0.072,0.07,0.051,0.043,0.033,0.014,0.017




Testing Expected (gold) outcome (what would the model look at to get to the expected outcome)
Sample: 10
Index: 1564
This prediction is correct
Input -> output (gold): 'seçilmek V;OBLIG;SG;1;NEG;PST;INTR;LGSPEC03' -> 'seçilmemeli miymişim'


Attributing with input_x_gradient...: 100%|█████████████████████████████| 24/24 [00:16<00:00,  1.36it/s]


,s,e,Ã,§,i,l,m,e,m,e,l,i,,m,i,y,m,i,Å,,i,m,</s>
s,0.023,0.05,0.055,0.052,0.023,0.022,0.017,0.018,0.01,0.01,0.008,0.016,0.021,0.012,0.009,0.01,0.01,0.005,0.009,0.024,0.005,0.012,0.011
e,0.026,0.073,0.039,0.031,0.025,0.03,0.044,0.014,0.015,0.01,0.007,0.012,0.02,0.016,0.005,0.015,0.014,0.006,0.017,0.01,0.007,0.01,0.008
Ã,0.019,0.033,0.145,0.036,0.037,0.048,0.018,0.021,0.015,0.014,0.02,0.031,0.031,0.019,0.01,0.012,0.034,0.017,0.014,0.017,0.015,0.022,0.031
§,0.017,0.047,0.046,0.069,0.069,0.057,0.029,0.041,0.026,0.026,0.022,0.021,0.022,0.024,0.022,0.028,0.019,0.017,0.027,0.012,0.013,0.021,0.02
i,0.021,0.027,0.033,0.05,0.053,0.028,0.031,0.023,0.02,0.019,0.029,0.008,0.008,0.02,0.01,0.023,0.009,0.006,0.009,0.006,0.014,0.025,0.009
l,0.014,0.015,0.069,0.019,0.074,0.041,0.039,0.015,0.018,0.013,0.018,0.012,0.009,0.01,0.007,0.009,0.019,0.007,0.02,0.022,0.019,0.026,0.007
m,0.007,0.013,0.021,0.022,0.025,0.024,0.063,0.038,0.036,0.031,0.019,0.013,0.014,0.014,0.018,0.013,0.009,0.012,0.013,0.008,0.014,0.017,0.016
e,0.007,0.019,0.015,0.018,0.021,0.021,0.014,0.028,0.024,0.022,0.021,0.017,0.014,0.018,0.012,0.011,0.008,0.006,0.01,0.008,0.009,0.011,0.006
k,0.019,0.033,0.05,0.058,0.052,0.036,0.027,0.059,0.028,0.037,0.026,0.02,0.022,0.026,0.032,0.03,0.033,0.018,0.022,0.012,0.016,0.025,0.024
,0.024,0.035,0.029,0.02,0.02,0.025,0.018,0.02,0.041,0.016,0.032,0.017,0.019,0.018,0.015,0.018,0.021,0.009,0.019,0.012,0.013,0.014,0.012


In [13]:
header_names = ['lemma', 'labels', 'features']

turkish_train = pd.read_csv('./data/tur_large.train', sep='\t', names=header_names)

In [14]:
turkish_train.head()

,lemma,labels,features
0,masör,masörlerdim,N;ARGNO1S;PL;PST
1,kışkırtmak,kışkırtmış mıyım,V;SG;1;POS;PST;INTR;LGSPEC03
2,küçük,küçük değildiler,ADJ;DECL;PL;3;NEG;PST;LGSPEC01
3,Yahudi,Yahudimizin,N;GEN;SG;PSS1P
4,bitirttirmek,bitirttirecek olacak mıymışım,V;PROSP;SG;1;POS;FUT;INTR;LGSPEC03


In [38]:
filtered_turkish = turkish_train[
    turkish_train['features'].str.startswith('N;')  # Ensure it is a noun
  & turkish_train['features'].str.contains('PL;')]  # And plural
print(filtered_turkish.size)
filtered_turkish.head()

1803


,lemma,labels,features
0,masör,masörlerdim,N;ARGNO1S;PL;PST
28,pipet,pipetlerimiz,N;NOM;PL;PSS1P
36,göt,götlerinin,N;GEN;PL;PSS3S
57,bozayı,bozayılarınızdan,N;ABL;PL;PSS2P
82,hırsız,hırsızlarımızı,N;ACC;PL;PSS1P


In [35]:
completed_turkish = predict_on_data(filtered_turkish.head(), model_tur_finetuned)

            Expected        Predicted
0        masörlerdim    masörlerinden
28      pipetlerimiz    pipetlerinden
36        götlerinin      götlerinden
57  bozayılarınızdan     bozayılar mı
82    hırsızlarımızı  hırsızlarınızda


In [41]:
attributions = inseq_on_predictions(completed_turkish, inseq_tur_finetuned, expected=False)

Testing Predicted (pred) outcome (what did the model take from the input during generation)
Sample: 1
Index: 0
This prediction is incorrect
Input -> output (pred): 'masör N;ARGNO1S;PL;PST' -> 'masörlerinden'
The expected or gold output would be 'masörlerdim'


Attributing with input_x_gradient...: 100%|█████████████████████████████| 16/16 [00:08<00:00,  1.84it/s]


,m,a,s,Ã,¶,r,l,e,r,i,n,d,e,n,</s>
m,0.102,0.078,0.072,0.044,0.077,0.025,0.022,0.035,0.029,0.094,0.035,0.036,0.029,0.042,0.054
a,0.069,0.106,0.098,0.038,0.053,0.051,0.035,0.055,0.069,0.064,0.041,0.038,0.018,0.028,0.041
s,0.05,0.04,0.04,0.04,0.024,0.053,0.025,0.031,0.026,0.022,0.018,0.015,0.02,0.03,0.012
Ã,0.035,0.07,0.061,0.092,0.04,0.05,0.043,0.016,0.063,0.031,0.036,0.023,0.036,0.035,0.033
¶,0.06,0.134,0.106,0.136,0.104,0.102,0.051,0.029,0.152,0.065,0.034,0.026,0.068,0.035,0.087
r,0.034,0.086,0.14,0.168,0.064,0.099,0.074,0.017,0.073,0.074,0.023,0.038,0.11,0.04,0.034
,0.035,0.085,0.042,0.039,0.053,0.083,0.079,0.025,0.017,0.021,0.045,0.038,0.041,0.019,0.018
N,0.016,0.013,0.016,0.016,0.015,0.036,0.02,0.021,0.012,0.018,0.04,0.012,0.012,0.013,0.013
;,0.039,0.034,0.018,0.024,0.047,0.033,0.05,0.018,0.014,0.023,0.032,0.031,0.025,0.031,0.018
A,0.014,0.007,0.008,0.007,0.007,0.023,0.011,0.006,0.015,0.008,0.016,0.005,0.015,0.009,0.008




Testing Predicted (pred) outcome (what did the model take from the input during generation)
Sample: 2
Index: 28
This prediction is incorrect
Input -> output (pred): 'pipet N;NOM;PL;PSS1P' -> 'pipetlerinden'
The expected or gold output would be 'pipetlerimiz'


Attributing with input_x_gradient...: 100%|█████████████████████████████| 15/15 [00:07<00:00,  1.80it/s]


...

Attributing with input_x_gradient...: 100%|█████████████████████████████| 16/16 [00:08<00:00,  1.70it/s]


,b,o,z,a,y,Ä,±,l,a,r,,m,Ä,±,</s>
b,0.089,0.108,0.144,0.154,0.164,0.139,0.034,0.07,0.082,0.081,0.106,0.022,0.032,0.018,0.019
o,0.065,0.061,0.063,0.09,0.09,0.109,0.067,0.044,0.045,0.063,0.076,0.022,0.017,0.02,0.024
z,0.122,0.088,0.075,0.036,0.03,0.035,0.021,0.019,0.044,0.06,0.039,0.034,0.028,0.021,0.017
a,0.033,0.08,0.057,0.057,0.045,0.036,0.018,0.01,0.014,0.029,0.02,0.016,0.022,0.026,0.034
y,0.028,0.055,0.062,0.034,0.029,0.037,0.026,0.034,0.019,0.026,0.023,0.03,0.023,0.022,0.038
Ä,0.043,0.031,0.032,0.03,0.029,0.057,0.027,0.029,0.038,0.035,0.032,0.035,0.021,0.025,0.026
±,0.07,0.045,0.042,0.095,0.087,0.075,0.048,0.044,0.034,0.027,0.056,0.029,0.024,0.025,0.031
,0.065,0.092,0.085,0.037,0.062,0.072,0.038,0.151,0.075,0.016,0.062,0.091,0.018,0.027,0.029
N,0.05,0.013,0.012,0.008,0.01,0.013,0.012,0.025,0.018,0.015,0.015,0.032,0.032,0.03,0.015
;,0.047,0.05,0.04,0.026,0.027,0.02,0.018,0.029,0.023,0.022,0.034,0.019,0.024,0.041,0.015




Testing Predicted (pred) outcome (what did the model take from the input during generation)
Sample: 5
Index: 82
This prediction is incorrect
Input -> output (pred): 'hırsız N;ACC;PL;PSS1P' -> 'hırsızlarınızda'
The expected or gold output would be 'hırsızlarımızı'


Attributing with input_x_gradient...: 100%|█████████████████████████████| 21/21 [00:11<00:00,  1.70it/s]


,h,Ä,±,r,s,Ä,±,z,l,a,r,Ä,±,n,Ä,±,z,d,a,</s>
h,0.083,0.077,0.059,0.14,0.073,0.037,0.03,0.057,0.035,0.027,0.038,0.084,0.022,0.039,0.053,0.023,0.027,0.029,0.015,0.018
Ä,0.054,0.128,0.074,0.059,0.053,0.052,0.059,0.033,0.025,0.027,0.019,0.037,0.057,0.027,0.018,0.053,0.019,0.021,0.017,0.028
±,0.054,0.06,0.078,0.082,0.075,0.077,0.02,0.027,0.04,0.014,0.016,0.015,0.033,0.014,0.023,0.017,0.012,0.011,0.013,0.023
r,0.132,0.097,0.048,0.093,0.052,0.047,0.106,0.102,0.029,0.055,0.022,0.035,0.038,0.038,0.033,0.037,0.017,0.017,0.016,0.025
s,0.02,0.03,0.047,0.041,0.07,0.067,0.024,0.036,0.02,0.027,0.031,0.017,0.02,0.018,0.012,0.017,0.013,0.015,0.023,0.025
Ä,0.021,0.031,0.037,0.045,0.036,0.108,0.035,0.025,0.025,0.034,0.032,0.02,0.023,0.021,0.025,0.022,0.018,0.014,0.024,0.018
±,0.024,0.044,0.092,0.043,0.044,0.032,0.042,0.027,0.048,0.023,0.033,0.044,0.053,0.039,0.029,0.032,0.05,0.057,0.038,0.018
z,0.033,0.063,0.048,0.034,0.067,0.053,0.04,0.081,0.054,0.041,0.053,0.036,0.063,0.034,0.047,0.064,0.04,0.042,0.034,0.033
,0.059,0.067,0.059,0.113,0.123,0.093,0.019,0.079,0.142,0.1,0.028,0.053,0.032,0.056,0.031,0.031,0.027,0.033,0.07,0.028
N,0.019,0.014,0.016,0.014,0.018,0.017,0.019,0.024,0.032,0.042,0.017,0.014,0.022,0.017,0.012,0.012,0.013,0.025,0.026,0.017
